In [2]:
from langchain.prompts import PromptTemplate 
from langchain.chains import RetrievalQA 
from langchain.embeddings import HuggingFaceEmbeddings 
from langchain.vectorstores import Pinecone 
import pinecone 
from langchain.document_loaders import PyPDFLoader, DirectoryLoader 
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from langchain.llms import CTransformers 
from tqdm.autonotebook import tqdm



d:\break\Django\nlp-env\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
# Extract data from pdf 

def load_pdf(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader,
    )

    documents = loader.load()

    return documents

In [4]:
extractd_data = load_pdf('data/')

In [5]:
extractd_data[:5]

[Document(page_content='', metadata={'source': 'data\\Medical_book.pdf', 'page': 0}),
 Document(page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION', metadata={'source': 'data\\Medical_book.pdf', 'page': 1}),
 Document(page_content='TheGALE\nENCYCLOPEDIA\nofMEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nA-B1', metadata={'source': 'data\\Medical_book.pdf', 'page': 2}),
 Document(page_content='STAFF\nJacqueline L. Longe, Project Editor\nDeirdre S. Blanchfield, Associate Editor\nChristine B. Jeryan, Managing Editor\nDonna Olendorf, Senior Editor\nStacey Blachford, Associate Editor\nKate Kretschmann, Melissa C. McDade, Ryan\nThomason, Assistant Editors\nMark Springer, Technical Specialist\nAndrea Lopeman, Programmer/Analyst\nBarbara J. Yarrow, Manager, Imaging and Multimedia\nContent\nRobyn V . Young, Project Manager, Imaging and\nMultimedia Content\nDean Dauphinais, Senior Editor, Imaging and\nMultimedia Content\nKelly 

In [6]:
# Create a text chunks 

def text_chunks(data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )

    chunks = text_splitter.split_documents(data)

    return chunks

In [7]:
text_chunks = text_chunks(extractd_data)
print(f'Length of the chunk', len(text_chunks))

Length of the chunk 7020


In [8]:
# Download embedding model 

def download_hugging_face_embedding():
    embeddings = HuggingFaceEmbeddings(
        # model_name="sentence-transformers/all-mpnet-base-v2",
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        # device="cuda",
    )
    return embeddings

In [9]:
embeddings = download_hugging_face_embedding()

d:\break\Django\nlp-env\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\break\Django\nlp-env\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [10]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [12]:
query_result = embeddings.embed_query('Hyalo')
print( len(query_result))

384


In [15]:
from pinecone import Pinecone
from dotenv import load_dotenv
import os
from langchain.vectorstores import Pinecone as PineconeStore
from langchain_pinecone import PineconeVectorStore

load_dotenv()
API_KEY = os.getenv("PINECONE_API_KEY")

pc = Pinecone(api_key=API_KEY)
index = pc.Index("chatbot")

docsearch = PineconeStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name='chatbot')


In [16]:
docsearch = PineconeVectorStore.from_existing_index('chatbot', embeddings)
query = 'What are allergies'

docs = docsearch.similarity_search(query, k=5)

print('Result', docs)


Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drugs commonly ca

In [17]:
# Create a prompt template

Prompt_template = '''
Use the following pieces of information to answer the question:
if you don't have enough information to answer the question, please respond with "I don't have enough information to answer the question."

Context: {context}
Question: {question}

Only response with helpful information.
Answer:
'''

In [28]:
PROMPT = PromptTemplate(template=Prompt_template, input_variables=['context', 'question'])
chain_type_kwargs = {'prompt': PROMPT}

In [29]:
!pip install ctransformers

In [30]:
llm = CTransformers(
    model = 'model/llama-2-7b-chat.bin',
    model_type = 'llama',
    config = {
        'max_new_tokens':512,
        'temperature':0.5,
    }
)

In [31]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',  # Replace with a valid chain type
    retriever=docsearch.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)


In [34]:
while True:
    user_input = input(f'Input prompt:')
    if user_input == 'exit':
        break
    response = qa({
        'query': user_input,
    })
    print("Response: ", response['result'])

Response:  I don't have enough information to answer the question.
Response:  I don't have enough information to answer the question.
Response:  I don't have enough information to answer the question. The passage does not provide sufficient context or details about the patient's condition or medical history to determine a prognosis for their heart arrhythmia. Additionally, the passage does not mention any specific symptoms or test results that could help estimate the severity of the condition. Without more information, it is impossible to provide an accurate prognosis.
Response:  Acne is a common skin disorder characterized by inflamed pores on the face, chest, and back caused by clogged oil, dead skin cells, and bacteria. It occurs when the sebaceous glands become inflamed, leading to pimples, whiteheads, and blackheads. Acne can cause emotional distress and affect a person's self-esteem and quality of life. Treatment options include topical creams and gels, oral antibiotics, and horm